In [1]:
# PyTorch
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Scheduler - OneCycleLR, CosineAnnealingLR
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR

# PyTorch Lightning
import lightning as L

# wandb
import wandb

# Ax - Hyperparameter Optimization
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.service.utils.report_utils import exp_to_df
from ax.utils.notebook.plotting import init_notebook_plotting, render

# Split the data into training and test sets
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import polars as pl

In [2]:
import os
os.environ["WANDB_SILENT"] = "true"

import warnings
warnings.filterwarnings('ignore')

In [3]:
L.seed_everything(42)

Seed set to 42


42

In [4]:
df_grf = pl.read_parquet("../data/grf.parquet")
df_grf_int = pl.read_parquet("../data/grf_int.parquet")

In [5]:
n_samples = df_grf["group"].n_unique()
n_samples

10000

In [6]:
print(df_grf, df_grf_int)

shape: (10_000_000, 3)
┌───────┬──────────┬───────┐
│ x     ┆ grf      ┆ group │
│ ---   ┆ ---      ┆ ---   │
│ f64   ┆ f64      ┆ u64   │
╞═══════╪══════════╪═══════╡
│ 0.0   ┆ 0.201082 ┆ 0     │
│ 0.001 ┆ 0.201827 ┆ 0     │
│ 0.002 ┆ 0.202477 ┆ 0     │
│ 0.003 ┆ 0.203033 ┆ 0     │
│ 0.004 ┆ 0.203495 ┆ 0     │
│ …     ┆ …        ┆ …     │
│ 0.996 ┆ 0.304625 ┆ 9999  │
│ 0.997 ┆ 0.303263 ┆ 9999  │
│ 0.998 ┆ 0.301747 ┆ 9999  │
│ 0.999 ┆ 0.300075 ┆ 9999  │
│ 1.0   ┆ 0.298249 ┆ 9999  │
└───────┴──────────┴───────┘ shape: (1_000_000, 3)
┌──────┬───────────┬───────┐
│ y    ┆ grf_int   ┆ group │
│ ---  ┆ ---       ┆ ---   │
│ f64  ┆ f64       ┆ u64   │
╞══════╪═══════════╪═══════╡
│ 0.0  ┆ 0.0       ┆ 0     │
│ 0.01 ┆ 0.002035  ┆ 0     │
│ 0.02 ┆ 0.004055  ┆ 0     │
│ 0.03 ┆ 0.005968  ┆ 0     │
│ 0.04 ┆ 0.007688  ┆ 0     │
│ …    ┆ …         ┆ …     │
│ 0.96 ┆ -0.005045 ┆ 9999  │
│ 0.97 ┆ -0.00228  ┆ 9999  │
│ 0.98 ┆ 0.000709  ┆ 9999  │
│ 0.99 ┆ 0.0038    ┆ 9999  │
│ 1.0  ┆ 0.006852  ┆ 9999  

In [7]:
df_grf = df_grf.filter(pl.col("x").is_in([round(x * 0.01, 2) for x in range(101)]))
print(df_grf)

shape: (1_000_000, 3)
┌──────┬──────────┬───────┐
│ x    ┆ grf      ┆ group │
│ ---  ┆ ---      ┆ ---   │
│ f64  ┆ f64      ┆ u64   │
╞══════╪══════════╪═══════╡
│ 0.0  ┆ 0.201082 ┆ 0     │
│ 0.01 ┆ 0.204293 ┆ 0     │
│ 0.02 ┆ 0.198184 ┆ 0     │
│ 0.03 ┆ 0.183089 ┆ 0     │
│ 0.04 ┆ 0.15959  ┆ 0     │
│ …    ┆ …        ┆ …     │
│ 0.96 ┆ 0.261699 ┆ 9999  │
│ 0.97 ┆ 0.289567 ┆ 9999  │
│ 0.98 ┆ 0.306218 ┆ 9999  │
│ 0.99 ┆ 0.309601 ┆ 9999  │
│ 1.0  ┆ 0.298249 ┆ 9999  │
└──────┴──────────┴───────┘


In [8]:
x = df_grf.filter(pl.col("group") == 0)["x"].to_numpy()
y = df_grf_int.group_by("group", maintain_order=True).agg(pl.col("y"))["y"].explode().to_numpy().reshape(n_samples, -1)
grfs = df_grf.group_by("group", maintain_order=True).agg(pl.col("grf"))["grf"].explode().to_numpy().reshape(n_samples, -1)
grf_ints = df_grf_int.group_by("group", maintain_order=True).agg(pl.col("grf_int"))["grf_int"].explode().to_numpy().reshape(n_samples, -1)

y = y.astype(np.float32)
grfs = grfs.astype(np.float32)
grf_ints = grf_ints.astype(np.float32)

print(f"x: {x.shape}, y: {y.shape}")
print(f"grfs: {grfs.shape}, grf_ints: {grf_ints.shape}")

x: (100,), y: (10000, 100)
grfs: (10000, 100), grf_ints: (10000, 100)


## DeepONet from Scratch

$$
G: u \in C[\mathcal{D}] \rightarrow G(u) \in C[\mathcal{R}] \quad \text{where } \mathcal{D}, \mathcal{R} \text{ are compact}
$$
$$
u(x) \overset{G}{\longrightarrow} G(u)(y) = \int_0^y u(x) dx
$$

In [9]:
n_train = int(0.8 * n_samples)
n_val = int(0.1 * n_samples)
n_test = n_samples - n_train - n_val

grf_train = grfs[:n_train]
grf_val = grfs[n_train:n_train + n_val]
grf_test = grfs[n_train + n_val:]

y_train = y[:n_train]
y_val = y[n_train:n_train + n_val]
y_test = y[n_train + n_val:]

grf_int_train = grf_ints[:n_train]
grf_int_val = grf_ints[n_train:n_train + n_val]
grf_int_test = grf_ints[n_train + n_val:]

In [10]:
class IntegralData(Dataset):
    def __init__(self, grf, y, grf_int):
        self.grf = torch.tensor(grf)
        self.y = torch.tensor(y)
        self.grf_int = torch.tensor(grf_int)

    def __len__(self):
        return len(self.grf)

    def __getitem__(self, idx):
        return self.grf[idx], self.y[idx], self.grf_int[idx]

In [11]:
ds_train = IntegralData(grf_train, y_train, grf_int_train)
ds_val = IntegralData(grf_val, y_val, grf_int_val)
ds_test = IntegralData(grf_test, y_test, grf_int_test)

In [12]:
class DeepONetScratch(nn.Module):
    def __init__(self, hparams):
        super().__init__()
        
        num_input = hparams["num_input"]
        num_branch = hparams["num_branch"]
        num_output = hparams["num_output"]
        dim_output = hparams["dim_output"]
        hidden_size = hparams["hidden_size"]
        hidden_depth = hparams["hidden_depth"]

        branch_net = [nn.Linear(num_input, hidden_size), nn.GELU()]
        for i in range(hidden_depth-1):
            branch_net.append(nn.Linear(hidden_size, hidden_size))
            branch_net.append(nn.GELU())
        branch_net.append(nn.Linear(hidden_size, num_branch))
        self.branch_net = nn.Sequential(*branch_net)

        trunk_net = [nn.Linear(dim_output, hidden_size), nn.GELU()]
        for _ in range(hidden_depth-1):
            trunk_net.append(nn.Linear(hidden_size, hidden_size))
            trunk_net.append(nn.GELU())
        trunk_net.append(nn.Linear(hidden_size, num_branch))
        self.trunk_net = nn.Sequential(*trunk_net)
        
        self.bias = nn.Parameter(torch.randn(num_output), requires_grad=True)

    def forward(self, u, y):
        l = y.shape[1]
        branch_out = self.branch_net(u)
        trunk_out = torch.stack([self.trunk_net(y[:, i:i+1]) for i in range(l)], dim=2)
        pred = torch.einsum("bp,bpl->bl", branch_out, trunk_out) + self.bias
        return pred

In [13]:
def train(model, optimizer, scheduler, train_loader, val_loader, epochs, device):
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for u, y, Guy in train_loader:
            u, y, Guy = u.to(device), y.to(device), Guy.to(device)
            optimizer.zero_grad()
            pred = model(u, y)
            loss = F.mse_loss(pred, Guy)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for u, y, Guy in val_loader:
                u, y, Guy = u.to(device), y.to(device), Guy.to(device)
                pred = model(u, y)
                loss = F.mse_loss(pred, Guy)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        
        scheduler.step()

        wandb.log({"train_loss": train_loss, "val_loss": val_loss, "epoch": epoch+1})

In [14]:
def evaluate(model, test_loader, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for u, y, Guy in test_loader:
            u, y, Guy = u.to(device), y.to(device), Guy.to(device)
            pred = model(u, y)
            loss = F.mse_loss(pred, Guy)
            test_loss += loss.item()
    test_loss /= len(test_loader)
    return test_loss

In [15]:
dl_train = DataLoader(ds_train, batch_size=500, shuffle=True)
dl_val = DataLoader(ds_val, batch_size=500)
dl_test = DataLoader(ds_test, batch_size=500)

## Ax for hyperparameter tuning

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def evaluate_model(parameters):
    hparams = {
        "num_input": parameters.get("num_input", 100),
        "num_branch": parameters.get("num_branch", 10),
        "num_output": parameters.get("num_output", 100),
        "dim_output": parameters.get("dim_output", 1),
        "hidden_size": parameters.get("hidden_size", 40),
        "hidden_depth": parameters.get("hidden_depth", 3),
        "learning_rate": parameters.get("learning_rate", 1e-2),
        "batch_size": parameters.get("batch_size", 500),
        "epochs": parameters.get("epochs", 200)
    }
    model = DeepONetScratch(hparams)
    
    wandb.init(project="DeepONet", config=hparams)
    
    optimizer = optim.Adam(model.parameters(), lr=hparams["learning_rate"])
    scheduler = OneCycleLR(optimizer, max_lr=hparams["learning_rate"], epochs=hparams["epochs"], steps_per_epoch=len(dl_train) // hparams["batch_size"] + 1)
    train(model, optimizer, scheduler, dl_train, dl_val, hparams["epochs"], device)
    test_loss = evaluate(model, dl_test, device)
    
    wandb.log({"test_loss": test_loss})
    wandb.finish()

    print(test_loss)
    
    return test_loss * 1e+5

cuda


In [17]:
ax_client = AxClient(verbose_logging=False)

In [18]:
ax_client.create_experiment(
    name="DeepONet-Tuning",
    parameters=[
        {
            "name": 'num_input',
            "type": 'fixed',
            "value": 100,
        },
        {
            "name": 'num_branch',
            "type": 'choice',
            "values": [10, 20, 30, 40],
            "value_type": "int",
            "is_ordered": True,
            "sort_values": False,
        },
        {
            "name": 'num_output',
            "type": 'fixed',
            "value": 100,
        },
        {
            "name": 'dim_output',
            "type": 'fixed',
            "value": 1,
        },
        {
            "name": 'hidden_size',
            "type": 'choice',
            "values": [40, 80, 120, 160],
            "value_type": "int",
            "is_ordered": True,
            "sort_values": False,
        },
        {
            "name": 'hidden_depth',
            "type": 'choice',
            "values": [2, 3, 4],
            "value_type": "int",
            "is_ordered": True,
            "sort_values": False,
        },
        {
            "name": 'learning_rate',
            "type": 'range',
            "bounds": [1e-3, 2e-2],
            "log_scale": True,
        },
        {
            "name": 'batch_size',
            "type": 'fixed',
            "value": 500,
        },
        {
            "name": 'epochs',
            "type": 'fixed',
            "value": 200,
        },
    ],
    objectives={"evaluate_model": ObjectiveProperties(minimize=True)},
)

[INFO 03-23 23:44:06] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter learning_rate. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-23 23:44:06] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[FixedParameter(name='num_input', parameter_type=INT, value=100), ChoiceParameter(name='num_branch', parameter_type=INT, values=[10, 20, 30, 40], is_ordered=True, sort_values=False), FixedParameter(name='num_output', parameter_type=INT, value=100), FixedParameter(name='dim_output', parameter_type=INT, value=1), ChoiceParameter(name='hidden_size', parameter_type=INT, values=[40, 80, 120, 160], is_ordered=True, sort_values=False), ChoiceParameter(name='hidden_depth', parameter_type=INT, values=[2, 3, 4], is_ordered=True, sort_values=False), RangeParameter(name='learning_rate', parameter_type=FLOAT, range=[0.001, 0.02], log_scale=True), 

In [19]:
for _ in range(100):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate_model(parameters))

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


6.29210376246192e-06


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


6.817745088483207e-05


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

7.943242962937802e-05


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


2.2760252250009216e-05


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


0.021407472901046276


In [20]:
best_parameters, values = ax_client.get_best_parameters()
best_parameters

{'num_branch': 20,
 'hidden_size': 120,
 'hidden_depth': 4,
 'learning_rate': 0.005548637705383897,
 'num_input': 100,
 'num_output': 100,
 'dim_output': 1,
 'batch_size': 500,
 'epochs': 200}

In [21]:
ax_client.generation_strategy.trials_as_df

[INFO 03-24 03:38:49] ax.modelbridge.generation_strategy: Note that parameter values in dataframe are rounded to 2 decimal points; the values in the dataframe are thus not the exact ones suggested by Ax in trials.


,Generation Step,Generation Model,Trial Index,Trial Status,Arm Parameterizations
0,GenerationStep_0,Sobol,0,COMPLETED,"{'0_0': {'num_branch': 10, 'hidden_size': 120,..."
1,GenerationStep_0,Sobol,1,COMPLETED,"{'1_0': {'num_branch': 30, 'hidden_size': 80, ..."
2,GenerationStep_0,Sobol,2,COMPLETED,"{'2_0': {'num_branch': 30, 'hidden_size': 120,..."
3,GenerationStep_0,Sobol,3,COMPLETED,"{'3_0': {'num_branch': 10, 'hidden_size': 120,..."
4,GenerationStep_0,Sobol,4,COMPLETED,"{'4_0': {'num_branch': 30, 'hidden_size': 80, ..."
...,...,...,...,...,...
95,GenerationStep_1,BoTorch,95,COMPLETED,"{'95_0': {'num_branch': 40, 'hidden_size': 40,..."
96,GenerationStep_1,BoTorch,96,COMPLETED,"{'96_0': {'num_branch': 40, 'hidden_size': 160..."
97,GenerationStep_1,BoTorch,97,COMPLETED,"{'97_0': {'num_branch': 20, 'hidden_size': 160..."
98,GenerationStep_1,BoTorch,98,COMPLETED,"{'98_0': {'num_branch': 40, 'hidden_size': 40,..."


In [22]:
render(ax_client.get_optimization_trace())

In [23]:
ax_client.get_best_trial()

(99,
 {'num_branch': 20,
  'hidden_size': 120,
  'hidden_depth': 4,
  'learning_rate': 0.005548637705383897,
  'num_input': 100,
  'num_output': 100,
  'dim_output': 1,
  'batch_size': 500,
  'epochs': 200},
 ({'evaluate_model': 0.4888443210730218},
  {'evaluate_model': {'evaluate_model': 0.8415911472557052}}))

In [24]:
from ax.plot.scatter import interact_fitted, plot_objective_vs_constraints, tile_fitted
from ax.plot.slice import plot_slice

In [25]:
ax_model = ax_client.generation_strategy.model

In [26]:
render(plot_slice(ax_model, "learning_rate", "evaluate_model"))

In [27]:
best_param, _ = ax_client.get_best_parameters()
best_param

{'num_branch': 20,
 'hidden_size': 120,
 'hidden_depth': 4,
 'learning_rate': 0.005548637705383897,
 'num_input': 100,
 'num_output': 100,
 'dim_output': 1,
 'batch_size': 500,
 'epochs': 200}

In [31]:
render(ax_client.get_feature_importances())

In [33]:
ax_df = ax_client.get_trials_data_frame()
ax_df

[WARNING 03-24 09:23:11] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


,trial_index,arm_name,trial_status,generation_method,evaluate_model,num_branch,hidden_size,hidden_depth,learning_rate,num_input,num_output,dim_output,batch_size,epochs
0,0,0_0,COMPLETED,Sobol,0.629210,10,120,4,0.014092,100,100,1,500,200
1,1,1_0,COMPLETED,Sobol,6.817745,30,80,3,0.003648,100,100,1,500,200
2,2,2_0,COMPLETED,Sobol,436.809356,30,120,3,0.009688,100,100,1,500,200
3,3,3_0,COMPLETED,Sobol,8.580369,10,120,2,0.007530,100,100,1,500,200
4,4,4_0,COMPLETED,Sobol,199.146860,30,80,4,0.001706,100,100,1,500,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,95_0,COMPLETED,BoTorch,6.860256,40,40,4,0.008025,100,100,1,500,200
96,96,96_0,COMPLETED,BoTorch,970.015442,40,160,2,0.001903,100,100,1,500,200
97,97,97_0,COMPLETED,BoTorch,7.943243,20,160,3,0.007393,100,100,1,500,200
98,98,98_0,COMPLETED,BoTorch,2.276025,40,40,4,0.005028,100,100,1,500,200


In [34]:
# sort dataframe by evaluate_model
ax_df_sorted = ax_df.sort_values("evaluate_model")
ax_df_sorted

,trial_index,arm_name,trial_status,generation_method,evaluate_model,num_branch,hidden_size,hidden_depth,learning_rate,num_input,num_output,dim_output,batch_size,epochs
90,90,90_0,COMPLETED,BoTorch,0.216821,30,160,4,0.004755,100,100,1,500,200
41,41,41_0,COMPLETED,BoTorch,0.296143,20,120,4,0.008640,100,100,1,500,200
40,40,40_0,COMPLETED,BoTorch,0.367331,20,120,4,0.008002,100,100,1,500,200
72,72,72_0,COMPLETED,BoTorch,0.493383,30,120,4,0.005599,100,100,1,500,200
36,36,36_0,COMPLETED,BoTorch,0.573076,10,120,4,0.008450,100,100,1,500,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,77,77_0,COMPLETED,BoTorch,1900.860015,10,160,2,0.001294,100,100,1,500,200
99,99,99_0,COMPLETED,BoTorch,2140.747290,10,160,4,0.001000,100,100,1,500,200
65,65,65_0,COMPLETED,BoTorch,2202.260960,40,40,4,0.001000,100,100,1,500,200
5,5,5_0,COMPLETED,Sobol,2285.499591,10,40,2,0.019147,100,100,1,500,200


In [36]:
ax_client.get_best_trial()

(99,
 {'num_branch': 20,
  'hidden_size': 120,
  'hidden_depth': 4,
  'learning_rate': 0.005548637705383897,
  'num_input': 100,
  'num_output': 100,
  'dim_output': 1,
  'batch_size': 500,
  'epochs': 200},
 ({'evaluate_model': 0.4888443210730218},
  {'evaluate_model': {'evaluate_model': 0.8415911472557052}}))

In [40]:
ax_df[(ax_df["learning_rate"] > 0.0055) & (ax_df["learning_rate"] < 0.0056)]

,trial_index,arm_name,trial_status,generation_method,evaluate_model,num_branch,hidden_size,hidden_depth,learning_rate,num_input,num_output,dim_output,batch_size,epochs
20,20,20_0,COMPLETED,BoTorch,1.276865,20,120,4,0.005549,100,100,1,500,200
72,72,72_0,COMPLETED,BoTorch,0.493383,30,120,4,0.005599,100,100,1,500,200
82,82,82_0,COMPLETED,BoTorch,1.018398,30,120,4,0.005525,100,100,1,500,200


In [43]:
ax_df.iloc[20]["learning_rate"]

0.005548637705383897